## Load data and set db

In [3]:
%matplotlib inline
from tqdm import tqdm
from matplotlib import pyplot as plt
from domestic_journals import *
import pandas as pd
idx = pd.IndexSlice
glob = pd.read_csv('data/index.csv').set_index(['country_code','field_code','method_code','period'])['value']
cntrs = pd.read_csv('data/country.csv',index_col='country_code')
usedIndicators = ['euclid','cosine','GiniSimpson','weightGini','top3','instTOP3','shareEnglish','localShare']
db = DB_joinJournals('sqlite:///c:\\Users\\OP3202\\Documents\\Git\\GlobalizationPaper\\db\\180802_1611_AllJournals_ArReCp_2001_2017.sqlite')

## Get average growth of documents in the database 

In [ ]:
totyr = pd.read_sql_query('''
SELECT 
    p.name as year,
    sum(Articles) as Documents
FROM totalArticles ta
inner join periods p on ta.PeriodID = p.ID
where year >= 2005
group by year
''',con=db,index_col='year')
from scipy.stats.mstats import gmean
gm = gmean(1+totyr.pct_change().Documents.dropna())
totyr.Documents.iloc[0]*(gm)**12

## ANOVA - countries,disciplines,time

In [48]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
pd.options.display.float_format = '{:,.0%}'.format
from tqdm import tqdm 
# Calculating effect size
def anova_table(df,formula):
    model  = ols(formula, df).fit()
    aov = sm.stats.anova_lm(model, typ=2)
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov,model

ccodes = [col for col in glob.index.get_level_values('country_code').unique() if not col.startswith('_')]
flds_narrow = [col for col in glob.index.get_level_values('field_code').unique() if not col.startswith('bot')]
flds_broad = [col for col in glob.index.get_level_values('field_code').unique() if not col.startswith('top') or col == 'All']

d = {}

formula = 'value ~ C(country_code) + \
                   C(field_code) + \
                   C(period)'# + \
                #   C(country_code)*C(field_code)'# + \
                #    C(country_code)*C(period) + \
                #    C(field_code)*C(period) + \
                #    C(field_code)*C(period)*C(country_code)'
for method in tqdm(usedIndicators):
    df_narrow = glob.loc[idx[ccodes,flds_narrow,method,:]].to_frame().reset_index()
    anova_narrow, model_narrow = anova_table(df_narrow,formula)
    tbl_narrow = anova_narrow['omega_sq']
    tbl_narrow['r_squared'] = model_narrow.rsquared
    d[('narrow',method)] = tbl_narrow
    #d[('narrow','omega_sq',method)] = anova_table(df_narrow,formula)['omega_sq'].sum()
    #d[('narrow','r_sq',method)] = ols(formula,df_narrow).fit().rsquared

    df_broad = glob.loc[idx[ccodes,flds_broad,method,:]].to_frame().reset_index()
    anova_broad, model_broad = anova_table(df_broad,formula)
    tbl_broad = anova_broad['omega_sq']
    tbl_broad['r_squared'] = model_broad.rsquared
    d[('broad',method)] = tbl_broad

#anova_cntr_fld_time = pd.Series(d).unstack()#.stack(0).reorder_levels((1,0)).sort_index()
anova_cntr_fld_time = pd.DataFrame(d).stack(0).reorder_levels((1,0)).sort_index()

anova_cntr_fld_time.to_clipboard()
anova_cntr_fld_time



  0%|          | 0/8 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:04<00:34,  4.91s/it]

 25%|██▌       | 2/8 [00:09<00:29,  4.83s/it]

 38%|███▊      | 3/8 [00:13<00:23,  4.71s/it]

 50%|█████     | 4/8 [00:18<00:18,  4.57s/it]

 62%|██████▎   | 5/8 [00:22<00:13,  4.52s/it]

 75%|███████▌  | 6/8 [00:27<00:09,  4.53s/it]

 88%|████████▊ | 7/8 [00:31<00:04,  4.45s/it]

100%|██████████| 8/8 [00:35<00:00,  4.49s/it]


GiniSimpson  cosine  euclid  instTOP3  localShare  \
broad  C(country_code)          37%     48%     37%       44%         19%   
       C(field_code)            20%     16%     22%       17%         34%   
       C(period)                 1%      0%      0%        0%          2%   
       r_squared                58%     63%     59%       61%         56%   
narrow C(country_code)          55%     67%     58%       64%         37%   
       C(field_code)            14%      4%     11%        4%         23%   
       C(period)                 3%      0%      0%        1%          4%   
       r_squared                72%     72%     69%       69%         66%   

                        shareEnglish  top3  weightGini  
broad  C(country_code)           29%   32%         42%  
       C(field_code)             14%   25%         18%  
       C(period)                  1%    0%          1%  
       r_squared                 44%   57%         60%  
narrow C(country_code)           47%   53%         61%  
       C(field_code)             10%    9%          5%  
       C(period)                  2%    0%          2%  
       r_squared                 60%   63%         69%

In [47]:
anova_cntr_fld_time.loc[idx[:,'C(country_code)'],:]/anova_cntr_fld_time.loc[idx[:,'C(country_code)'],:]

,,GiniSimpson,cosine,euclid,instTOP3,localShare,shareEnglish,top3,weightGini
broad,C(country_code),37%,48%,37%,44%,19%,29%,32%,42%
narrow,C(country_code),55%,67%,58%,64%,37%,47%,53%,61%


In [24]:
glob.loc[idx[ccodes,:,'euclid',2017]].reset_index().groupby('field_code').apply(lambda x: len(x.country_code.unique()))

field_code
All                                         171
bot_AgriculturalAndBiological               132
bot_ArtsHumanities                           76
bot_BiochemistryGeneticsMolecularBiology    108
bot_BusinessManagementAccounting             78
bot_ChemicalEngineering                      77
bot_Chemistry                                96
bot_ComputerScience                          88
bot_DecisionSciences                         55
bot_Dentistry                                41
bot_EarthPlanetarySciences                   92
bot_EconomicsEconometricsFinance             73
bot_Energy                                   73
bot_Engineering                             102
bot_EnvironmentalScience                    111
bot_General                                   9
bot_HealthProfessions                        50
bot_ImmunologyMicrobiology                   90
bot_Materials                                93
bot_Mathematics                              91
bot_Medicine                 

In [ ]:
df = glob.loc[idx[ccodes,'All','euclid',2017]].to_frame().reset_index().drop(['field_code','method_code','period'],axis=1).merge(cntrs,left_on='country_code',right_index=True)

### K-Means clustering

In [ ]:
from sklearn.cluster import KMeans
dfkm = glob.loc[idx[
    [col for col in glob.index.get_level_values('country_code').unique() if not col.startswith('_')],
    [col for col in glob.index.get_level_values('field_code').unique() if col.startswith('top')],
    'euclid',
    2017]].unstack('field_code').dropna()

kmeans = KMeans(n_clusters=4)
kfit=kmeans.fit(dfkm,)
dfkm['cluster'] = kmeans.predict(dfkm)
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
mapdata = dfkm.reset_index()
world = pd.merge(world[['geometry', 'iso_a3']],mapdata,left_on='iso_a3',right_on='country_code')

f,ax = plt.subplots(1,figsize=(15,9))
world.plot(column='cluster',legend=False,ax=ax,cmap='Accent')
ax.set_axis_off()
plt.show()


## Hierarchical clustering

In [ ]:
{c[0]:c[1] for c in zip(dendr['ivl'],dendr['color_list'])}

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage,fcluster
from matplotlib import pyplot as plt
import geopandas as gpd

dfh= glob.loc[idx[
    [col for col in glob.index.get_level_values('country_code').unique() if not col.startswith('_')],
    [col for col in glob.index.get_level_values('field_code').unique() if col.startswith('top')],
    'euclid',
    2017]].unstack('field_code').dropna()
Z = linkage(dfh.values, 'ward')

labelList = dfh.index.get_level_values('country_code')
max_d = 1
fig,axs = plt.subplots(nrows=2,ncols=1,figsize=(20,12),gridspec_kw={'height_ratios': [4, 1]})

dendr = dendrogram(Z,
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=False,
            leaf_rotation=45.,  
            leaf_font_size=10.,
            color_threshold=max_d,
            ax=axs[1]
)

axs[1].axhline(y=max_d, c='k')

dfh['cluster'] = fcluster(Z, max_d, criterion='distance')
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.loc[world[world.name == 'France'].iloc[0].name,'iso_a3'] = 'FRA'
world.loc[world[world.name == 'Norway'].iloc[0].name,'iso_a3'] = 'NOR'

mapdata = dfh.reset_index()
world = pd.merge(world[['geometry', 'iso_a3']],mapdata,left_on='iso_a3',right_on='country_code')
world['cluster_color'] = world.country_code.map({c[0]:c[1] for c in zip(dendr['ivl'],dendr['color_list'])})
world.cluster_color.fillna('w',inplace=True)

world.plot(legend=False,ax=axs[0],color=world.cluster_color)
axs[0].set_axis_off()
#axs[1].set_axis_off()
fig.tight_layout()

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(dfh))
c


In [ ]:
countries = pd.DataFrame({
    2017:pd.read_sql_query('''
            select 
                c.name as Country,
                Sum(Articles) as Articles
            from ArticleCountries ac
            inner join countries c on c.ID = ac.FacetID
            inner join periods p on p.ID = ac.PeriodID
            where 
                p.name = 2017
            group by Country
            order by Articles DESC
        ''',con=db,index_col='Country').Articles,
    2005:pd.read_sql_query('''
            select 
                c.name as Country,
                Sum(Articles) as Articles
            from ArticleCountries ac
            inner join countries c on c.ID = ac.FacetID
            inner join periods p on p.ID = ac.PeriodID
            where 
                p.name = 2005
            group by Country
            order by Articles DESC
        ''',con=db,index_col='Country').Articles
    })

countries['imf2003'] = countries.index.map(pd.read_csv('data/country.csv',index_col='full_name').imf2003)
countries.groupby('imf2003').sum()[[2005,2017]]/countries.groupby('imf2003').sum()[[2005,2017]].sum()#.T.plot.bar(stacked=True)

In [ ]:
cntrgrowth = ((countries.loc[:,2017] - countries.loc[:,2005])/countries.loc[:,2005]).sort_values()

In [ ]:
fieldnumber = pd.read_sql_query('''
select 
    ie.broadFieldsNum as broadFieldsNumber,
    ie.narrowFieldsNum as narrowFieldsNumber,
    Sum(Articles) as Documents from totalArticles ta 
inner join 
    (select 
        *,
        (top_Life + top_Social + top_Physical + top_Health) as broadFieldsNum,
        (bot_General + bot_AgriculturalAndBiological + bot_ArtsHumanities +
        bot_BiochemistryGeneticsMolecularBiology + bot_BusinessManagementAccounting + 
        bot_ChemicalEngineering + bot_Chemistry + bot_ComputerScience + 
        bot_DecisionSciences + bot_EarthPlanetarySciences + bot_EconomicsEconometricsFinance +
        bot_Energy + bot_Engineering + bot_EnvironmentalScience + bot_ImmunologyMicrobiology +
        bot_Materials + bot_Mathematics + bot_Medicine + bot_Neuroscience + bot_Nursing + 
        bot_PharmacologyToxicologyPharmaceutics + bot_PhysicsAstronomy + bot_Psychology + 
        bot_SocialSciences + bot_Veterinary + bot_Dentistry + bot_HealthProfessions) as narrowFieldsNum
    from issns i
    ) ie
    ON ie.ID = ta.ISSNID
    group by ie.broadFieldsNum,
    ie.narrowFieldsNum
''',con=db)#.broadFieldsNum.hist(bins=3)
fieldshares = pd.DataFrame({
    'Broad':fieldnumber.groupby('broadFieldsNumber').Documents.sum()/fieldnumber.groupby('broadFieldsNumber').Documents.sum().sum(),
    'Narrow':fieldnumber.groupby('narrowFieldsNumber').Documents.sum()/fieldnumber.groupby('narrowFieldsNumber').Documents.sum().sum()
})
fieldshares.loc['More',:]=fieldshares.loc[4:,:].sum()

import matplotlib.ticker as mtick
fieldshares = fieldshares.loc[[0,1,2,3,'More'],:]*100
ax = fieldshares.T.plot.bar(title='Number of disciplines assigned to documents',stacked=True,cmap='gray')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())

## ANOVA

In [ ]:
glob

## Data Section

## All indicators show very similar results
### Full-Sample correlation


In [ ]:
corr_full = glob.unstack('method_code').corr()
corr_full.loc[usedIndicators,usedIndicators]

## Means and stds in all countries, disciplines by Economic status all indicators between 2015 and 2017

In [ ]:
d = {key:cntrs[key].unique() for key in ['imf2003']}

statuses = ['Advanced countries','Developing countries','Transition countries']
mdx = pd.MultiIndex.from_product([statuses,['mean','std']],names =['economic_status','stat'])
status = pd.DataFrame(index=mdx)
key = 'imf2003'
for field in ['All','top_Life','top_Physical','top_Social','top_Health']:
    for val in statuses:
        subdf = glob.loc[idx[cntrs[cntrs[key] == val].index,field,'euclid',[2015,2016,2017]]]
        status.loc[(val,'mean'),field] = subdf.mean()
        status.loc[(val,'std'),field] = subdf.std()

status

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,figsize=(10,6))
l = ['RUS','CHN']

lstyles = ['s-','v-','o-','P-','--']
for i in range(2):
    a = ax[i]
    a.set_ylim((0,1))
    a.set_title(l[i])
    #breakpoint()
    chnrus.xs(l[i]).T.plot(ax=a,style=lstyles,c='gray')#,marker='.',ls='-.',c='black')

In [ ]:
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
mapdata = pd.DataFrame(glob.loc[idx[:,'All','euclid',2017]])
world = pd.merge(world[['geometry', 'iso_a3']],mapdata,left_on='iso_a3',right_index=True)

f,ax = plt.subplots(1,figsize=(15,9))
#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="5%", pad=0.1)
world.plot(column='value',cmap='gray',legend=False,ax=ax)
ax.set_axis_off()
plt.show()


## Plotting LocalShares - main logic in data/paper_plots.py

In [ ]:
locals = pd.read_csv('localShares.csv',index_col='Country')
locals.localShare.plot(kind='bar',color='gray',figsize=(10,5))

## Output to excel

In [ ]:
writer = pd.ExcelWriter('paperTables.xlsx')
corr_full.loc[usedIndicators,usedIndicators].to_excel(writer,sheet_name='corr_full')
status.to_excel(writer,sheet_name='group_means_stds')
writer.save()

## Output CSVs as an appendix

In [ ]:
glob.to_csv('appendix/data.csv',header=True)
cntrs[cntrs.Type=='country'].imf2003.to_csv('appendix/countries.csv',header=True)

In [ ]:
dfh.dropna().values